# TrainingServingSkew
* Moniitors whether the alphas generated for inference are identical 
    to the alphas generated during training

In [ ]:
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("../InferenceAlphas/Alpha.ipynb");

In [ ]:
function read_training_alpha(alpha::String, split::String)
    uid = parse(Int, get_recommendee_username()) + 1
    content = nothing
    if occursin("Explicit", alpha) || alpha in ["UserAverage", "UserVariance"]
        @assert isnothing(content)
        content = "explicit"
    end
    if occursin("Implicit", alpha)
        @assert isnothing(content)
        content = "implicit"
    end
    if occursin("Ptw", alpha)
        @assert isnothing(content)
        content = "ptw"
    end
    @assert !isnothing(content)
    df = read_alpha(alpha, split, content)
    mask = df.user .== uid
    df =
        RatingsDataset(user = df.user[mask], item = df.item[mask], rating = df.rating[mask])
    df
end;

In [ ]:
function average_errors(alpha, split)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split)
    if length(training.item) == 0
        return 0
    end
    abs.(training.rating .- serving.rating[training.item])
end

function relative_errors(alpha, split)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split)
    if length(training.item) == 0
        return 0
    end
    ϵ = eps(Float64)
    abs.(abs.(training.rating) ./ (abs.(serving.rating[training.item]) .+ ϵ) .- 1)
end;

In [ ]:
function maximum_error(alpha, split)
    maximum(average_errors(alpha, split))
end

function average_abs_error(alpha, split)
    mean(average_errors(alpha, split))
end

function average_rel_error(alpha, split)
    mean(relative_errors(alpha, split))
end;

In [ ]:
function display_errors(alphas, split)
    alpha_padding = maximum(length.(alphas))
    number_padding = 16

    header_1 = rpad("Alpha", alpha_padding)
    header_2 = rpad("Avg Abs Error", number_padding)
    header_3 = rpad("Avg Rel Error", number_padding)
    header_4 = rpad("Max Error", number_padding)
    @info "$header_1 $header_2 $header_3 $header_4"

    for alpha in alphas
        avg_abs_error = rpad(average_abs_error(alpha, split), number_padding)
        avg_rel_error = rpad(average_rel_error(alpha, split), number_padding)
        max_error = rpad(maximum_error(alpha, split), number_padding)
        alpha = rpad(alpha, alpha_padding)
        @info "$alpha $avg_abs_error $avg_rel_error $max_error"
    end
end;

In [ ]:
potential_alphas = readdir(recommendee_alpha_basepath())
alphas = sort([
    x for
    x in potential_alphas if ispath("$(recommendee_alpha_basepath())/$(x)/alpha.jld2")
])
ensemble_alphas =
    sort(["$x$y" for x in ["Linear", "Nonlinear"] for y in ["Explicit", "Implicit", "Ptw"]])
inference_only_alphas = sort([
    ["ItemCFRelated"]
    ["BPR.$x$y" for x in ["neural", "tree"] for y in ["", ".ptw"]]
])
standard_alphas = [x for x in alphas if x ∉ ensemble_alphas && x ∉ inference_only_alphas];

In [ ]:
display_errors(standard_alphas, "training")
# ensemble alphas do not save training splits

In [ ]:
display_errors(standard_alphas, "validation")
println()
display_errors(ensemble_alphas, "validation")

In [ ]:
display_errors(standard_alphas, "test")
println()
display_errors(ensemble_alphas, "test")